# सत्र २ – न्यूनतम RAG पाइपलाइन

Foundry Local + sentence-transformers embeddings प्रयोग गरेर हल्का Retrieval-Augmented Generation पाइपलाइन निर्माण गर्नुहोस्।


### व्याख्या: निर्भरता स्थापना
यो पाइपलाइनका लागि न्यूनतम प्याकेजहरू स्थापना गर्दछ:
- `foundry-local-sdk` स्थानीय मोडेल व्यवस्थापनका लागि (यदि शुद्ध BASE_URL पथ प्रयोग गरिँदैन भने)।
- `openai` उपयुक्त SDK संरचनाहरूका लागि (केही उपयोगिताहरू)।
- `sentence-transformers` एम्बेडिङका लागि।
- `numpy` भेक्टर गणितका लागि।
पुन: चलाउन सुरक्षित; यदि वातावरण पहिले नै सन्तुष्ट छ भने छोड्न सकिन्छ।


# परिदृश्य
यो नोटबुकले पूर्ण रूपमा स्थानीय रूपमा चल्ने न्यूनतम Retrieval-Augmented Generation (RAG) पाइपलाइन निर्माण गर्दछ:
- Foundry Local मोडेलसँग जडान गर्दछ (SDK वा BASE_URL मार्फत स्वचालित रूपमा पत्ता लगाउँछ)।
- सानो इन-मेमोरी कागजात संग्रह सिर्जना गर्दछ र यसलाई Sentence Transformers को प्रयोग गरेर एम्बेड गर्दछ।
- पारदर्शिताका लागि साधारण भेक्टर समानता पुनःप्राप्ति (बाह्य इन्डेक्स बिना) कार्यान्वयन गर्दछ।
- धेरै HTTP फलब्याक मार्गहरू (`/v1/chat/completions`, `/v1/completions`, `/v1/responses`) मार्फत आधारभूत जेनेरेसन अनुरोधहरू जारी गर्दछ।
- `answer()` सहायक प्रदान गर्दछ, जसले प्रारम्भिक प्रयासहरू असफल हुँदा वैकल्पिक मोडेल रूपहरू पुन: प्रयास गर्दछ।

ठूलो कर्पस, स्थायी भेक्टर स्टोरहरू, वा मूल्याङ्कन मेट्रिक्समा विस्तार गर्नु अघि यसलाई एक डायग्नोस्टिक टेम्प्लेटको रूपमा प्रयोग गर्नुहोस् (RAG मूल्याङ्कन नोटबुक हेर्नुहोस्)।


In [5]:
# Install dependencies
!pip install -q foundry-local-sdk openai sentence-transformers numpy

### व्याख्या: कोर आयातहरू
एम्बेडिङ + स्थानीय अनुमानको लागि आवश्यक कोर पुस्तकालयहरू लोड गर्दछ:
- SentenceTransformer घनत्व भेक्टर एम्बेडिङको लागि।
- FoundryLocalManager (वैकल्पिक) स्थानीय सेवा व्यवस्थापन गर्न।
- OpenAI क्लाइन्ट परिचित वस्तु आकारहरूको लागि (यद्यपि पछि हामी HTTP सिधै प्रयोग गर्छौं)।


In [6]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI

### व्याख्या: खेलौना दस्तावेज संग्रह
डोमेन कथनहरूको सानो इन-मेमोरी सूची परिभाषित गर्दछ। पुनरावृत्ति छिटो र नियन्त्रणमा राख्छ ताकि ध्यान पाइपलाइन यान्त्रिकी (पुनःप्राप्ति + ग्राउन्डिङ) मा रहोस्, डाटा व्यवस्थापनमा होइन।


In [7]:
DOCS = [
    'Foundry Local provides an OpenAI-compatible local inference endpoint.',
    'Retrieval Augmented Generation improves answer grounding by injecting relevant context.',
    'Edge AI reduces latency and preserves privacy via local execution.',
    'Small Language Models can offer competitive quality with lower resource usage.',
    'Vector similarity search retrieves semantically relevant documents.'
]

### व्याख्या: जडान, मोडेल चयन र एम्बेडिङ सुरुवात
मजबुत जडान तर्क:
1. वैकल्पिक रूपमा स्पष्ट `BASE_URL` (शुद्ध HTTP पथ) प्रयोग गर्दछ, अन्यथा FoundryLocalManager मा फर्किन्छ।
2. `/v1/models` मा जाँच गर्दछ र सबैभन्दा उपयुक्त ठोस मोडेल आईडी चयन गर्दछ (ठ्याक्कै उपनाम > क्यानोनिकल परिवार > पहिलो उपलब्ध)।
3. पुन: प्रयास लूप लागू गर्दछ, जसमा `FOUNDRY_CONNECT_RETRIES` र ढिलाइ समायोज्य छ।
4. टय खेल सामग्रीको लागि SentenceTransformer एम्बेडिङ (सामान्यीकृत भेक्टरहरू) सुरुवात गर्दछ।
5. पुन: उत्पादनयोग्यता सुनिश्चित गर्न OpenAI SDK संस्करण समेट्छ।
यदि सेवा अनुपस्थित छ भने, क्र्यास नगरी यसलाई सुरु गर्न मार्गदर्शन प्रिन्ट गर्दछ।


In [12]:
import os, time, json, requests, re
# Native Foundry Local SDK preferred; fall back to explicit BASE_URL if provided
os.environ.setdefault('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
alias = os.getenv('FOUNDRY_LOCAL_ALIAS', os.getenv('TARGET_MODEL', 'phi-4-mini'))
base_url_env = os.getenv('BASE_URL', '').strip()
manager = None
client = None
endpoint = None

def _canonicalize(model_id: str) -> str:
    """Remove CUDA suffix and version tags from model name."""
    b = model_id.split(':')[0]
    return re.sub(r'-cuda.*', '', b)

try:
    if base_url_env:
        # Allow user override; normalize by removing trailing / and optional /v1
        root = base_url_env.rstrip('/')
        if root.endswith('/v1'):
            root = root[:-3]
        endpoint = root
        print(f'[INFO] Using explicit BASE_URL override: {endpoint}')
    else:
        from foundry_local import FoundryLocalManager
        manager = FoundryLocalManager(alias)
        # Manager endpoint already includes /v1 - remove it for our base
        raw_endpoint = manager.endpoint.rstrip('/')
        if raw_endpoint.endswith('/v1'):
            endpoint = raw_endpoint[:-3]
        else:
            endpoint = raw_endpoint
        print(f'[OK] Foundry Local manager endpoint: {manager.endpoint} | base={endpoint} | alias={alias}')
    
    # Probe models list (endpoint does NOT include /v1 here)
    models_resp = requests.get(endpoint + '/v1/models', timeout=5)
    models_resp.raise_for_status()
    payload = models_resp.json() if models_resp.headers.get('content-type','').startswith('application/json') else {}
    data = payload.get('data', []) if isinstance(payload, dict) else []
    ids = [m.get('id') for m in data if isinstance(m, dict)]
    
    # Select best matching model
    chosen = None
    if alias in ids:
        chosen = alias
    else:
        for mid in ids:
            if _canonicalize(mid) == _canonicalize(alias):
                chosen = mid
                break
    if not chosen and ids:
        chosen = ids[0]
    model_name = chosen or alias
    
    # Initialize OpenAI client
    from openai import OpenAI as _OpenAI
    client = _OpenAI(
        base_url=endpoint + '/v1',  # OpenAI client needs full base URL with /v1
        api_key=(getattr(manager, 'api_key', None) or os.getenv('API_KEY') or 'not-needed')
    )
    print(f'[OK] Model resolved: {model_name} (total_models={len(ids)})')
except Exception as e:
    print('[ERROR] Failed to initialize Foundry Local client:', e)
    client = None
    model_name = alias

# Expose BASE for downstream compatibility (without /v1)
BASE = endpoint

# Embeddings setup
embed_model_name = os.getenv('EMBED_MODEL', 'sentence-transformers/all-MiniLM-L6-v2')
try:
    from sentence_transformers import SentenceTransformer
    embedder = SentenceTransformer(embed_model_name)
    doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)
    print(f'[OK] Embedded {len(DOCS)} docs using {embed_model_name} shape={doc_emb.shape}')
except Exception as e:
    print('[ERROR] Embedding init failed:', e)
    embedder = None
    doc_emb = None

try:
    import openai as _openai
    openai_version = getattr(_openai, '__version__', 'unknown')
    print('OpenAI SDK version:', openai_version)
except Exception:
    openai_version = 'unknown'

if client is None:
    print('\nNEXT: Start/verify service then re-run this cell:')
    print('  foundry service start')
    print('  foundry model run phi-4-mini')
    print('  (optional) set BASE_URL=http://127.0.0.1:57127')

[OK] Foundry Local manager endpoint: http://127.0.0.1:59778/v1 | base=http://127.0.0.1:59778 | alias=phi-4-mini
[OK] Model resolved: deepseek-r1-distill-qwen-7b-cuda-gpu:0 (total_models=11)
[OK] Embedded 5 docs using sentence-transformers/all-MiniLM-L6-v2 shape=(5, 384)
OpenAI SDK version: 1.109.1


### व्याख्या: Retrieve Function (Vector Similarity)
`retrieve(query, k=3)` ले सोधपुछलाई एन्कोड गर्छ, कोसाइन समानता (सामान्यीकृत भेक्टरहरूमा डट प्रोडक्ट) गणना गर्छ, र शीर्ष-k डक इंडेक्सहरू फर्काउँछ। यो पारदर्शिताका लागि न्यूनतम र इन-मेमोरीमा रहन्छ।


In [9]:
def retrieve(query, k=3):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return sims.argsort()[::-1][:k]

### व्याख्या: SDK-आधारित जेनेरेसन र उत्तर सहायक
Foundry Local SDK र OpenAI-सँग मिल्दो क्लाइन्ट विधिहरू प्रयोग गरेर पुनःनिर्माण गरिएको छ, जसले कच्चा HTTP पोस्टहरूको सट्टा प्रयोग गर्दछ:
- प्राथमिक मार्ग: `client.chat.completions.create` (संरचित सन्देशहरू)।
- वैकल्पिक उपायहरू: `client.completions.create` (पुरानो प्रम्प्ट) त्यसपछि `client.responses.create` (सरलीकृत प्रतिक्रिया API)।
- वैकल्पिक मोडेल आईडीहरू (RAW बनाम हटाइएको ALT) सामान्यीकरण गरेर अनुकूलता विस्तार गर्दछ।
- `answer()` ले शीर्ष-k पुनःप्राप्त कागजातहरूबाट आधारित प्रम्प्ट निर्माण गर्दछ र प्रयासहरूको क्रमबद्ध ट्रेसहरू रेकर्ड गर्दछ।
यसले तर्कलाई पठनीय राख्छ जबकि OpenAI-सँग मिल्दो अन्तर्क्रियात्मक बिन्दुहरूको विकास हुँदै गर्दा सहज रूपमा कार्य गर्न सक्षम बनाउँछ।


In [14]:
# SDK-based generation (Foundry Local manager + OpenAI client methods)
import re, time, json

def _strip_model_name(name: str) -> str:
    """Strip CUDA suffix and version tags from model name."""
    base = name.split(':')[0]
    base = re.sub(r'-cuda.*', '', base)
    return base

# Use the actual resolved model name from connection cell
RAW_MODEL = model_name
ALT_MODEL = _strip_model_name(RAW_MODEL)

def _try_via_client(messages, prompt, model_id: str, max_tokens=220, temperature=0.2):
    """Try generating response using OpenAI client with multiple fallback routes."""
    attempts = []
    
    # 1. Try chat.completions endpoint (preferred for chat models)
    try:
        resp = client.chat.completions.create(
            model=model_id, 
            messages=messages, 
            max_tokens=max_tokens, 
            temperature=temperature
        )
        content = resp.choices[0].message.content
        attempts.append(('chat.completions', 200, (content or '')[:160]))
        if content and content.strip():
            return content, attempts
    except Exception as e:
        attempts.append(('chat.completions', None, str(e)[:160]))
    
    # 2. Try legacy completions endpoint
    try:
        comp = client.completions.create(
            model=model_id, 
            prompt=prompt, 
            max_tokens=max_tokens, 
            temperature=temperature
        )
        txt = comp.choices[0].text if comp.choices else ''
        attempts.append(('completions', 200, (txt or '')[:160]))
        if txt and txt.strip():
            return txt, attempts
    except Exception as e:
        attempts.append(('completions', None, str(e)[:160]))
    
    return None, attempts

def retrieve(query, k=3):
    """Retrieve top-k most similar documents using cosine similarity."""
    if embedder is None or doc_emb is None:
        raise RuntimeError("Embeddings not initialized.")
    q_emb = embedder.encode([query], normalize_embeddings=True)[0]
    scores = doc_emb @ q_emb
    idxs = np.argsort(scores)[::-1][:k]
    return idxs

def answer(query, k=3, max_tokens=220, temperature=0.2, try_alternate=True):
    """
    Answer a query using RAG pipeline:
    1. Retrieve relevant documents using vector similarity
    2. Generate grounded response using Foundry Local model via OpenAI SDK
    
    Args:
        query: User question
        k: Number of documents to retrieve
        max_tokens: Maximum tokens for generation
        temperature: Sampling temperature
        try_alternate: Whether to try alternate model name on failure
    
    Returns:
        Dictionary with query, answer, docs, context, route, and tried attempts
    """
    if client is None:
        raise RuntimeError('Model client not initialized. Re-run connection cell after starting Foundry Local.')
    if embedder is None or doc_emb is None:
        raise RuntimeError('Embeddings not initialized.')
    
    # Retrieve relevant documents
    idxs = retrieve(query, k=k)
    context = '\n'.join(f'Doc {i}: {DOCS[i]}' for i in idxs)
    
    # Construct grounded generation prompt
    system_content = 'Use ONLY provided context. If insufficient, say "I\'m not sure."'
    user_content = f'Context:\n{context}\n\nQuestion: {query}'
    messages = [
        {'role': 'system', 'content': system_content},
        {'role': 'user', 'content': user_content}
    ]
    prompt = f'System: {system_content}\n{user_content}\nAnswer:'
    
    # Try generation with primary model
    tried = []
    ans, attempts = _try_via_client(messages, prompt, RAW_MODEL, max_tokens=max_tokens, temperature=temperature)
    tried.append({'model': RAW_MODEL, 'attempts': attempts})
    
    if ans and ans.strip():
        return {
            'query': query, 
            'answer': ans.strip(), 
            'docs': idxs.tolist(), 
            'context': context, 
            'route': 'chat-first', 
            'tried': tried
        }
    
    # Try alternate model name if available
    if try_alternate and ALT_MODEL != RAW_MODEL:
        ans2, attempts2 = _try_via_client(messages, prompt, ALT_MODEL, max_tokens=max_tokens, temperature=temperature)
        tried.append({'model': ALT_MODEL, 'attempts': attempts2})
        if ans2 and ans2.strip():
            return {
                'query': query, 
                'answer': ans2.strip(), 
                'docs': idxs.tolist(), 
                'context': context, 
                'route': 'chat-alt', 
                'tried': tried
            }
    
    # All routes failed
    return {
        'query': query, 
        'answer': 'I\'m not sure. (All SDK routes failed)', 
        'docs': idxs.tolist(), 
        'context': context, 
        'route': 'failed', 
        'tried': tried
    }

print('[INFO] SDK generation mode active.')
print(f'       RAW_MODEL = {RAW_MODEL}')
print(f'       ALT_MODEL = {ALT_MODEL}')

[INFO] SDK generation mode active.
       RAW_MODEL = deepseek-r1-distill-qwen-7b-cuda-gpu:0
       ALT_MODEL = deepseek-r1-distill-qwen-7b


In [15]:
# Self-test cell: validates connectivity, embeddings, and answer() basic functionality (SDK mode)
import math, pprint

def rag_self_test(sample_query: str = 'Why use RAG with local inference?', expect_docs: int = 3):
    report = {'base': BASE, 'raw_model': RAW_MODEL, 'alt_model': ALT_MODEL}
    if not BASE:
        report['error'] = 'BASE not resolved'
        return report
    if embedder is None or doc_emb is None:
        report['error'] = 'Embeddings not initialized'
        return report
    if getattr(doc_emb, 'shape', (0,))[0] != len(DOCS):
        report['warning_embeddings'] = f"doc_emb count {getattr(doc_emb,'shape',('?'))} mismatch DOCS {len(DOCS)}"
    try:
        idxs = retrieve(sample_query, k=expect_docs)
        report['retrieved_indices'] = idxs.tolist() if hasattr(idxs, 'tolist') else list(idxs)
    except Exception as e:
        report['error_retrieve'] = str(e)
        return report
    try:
        ans = answer(sample_query, k=expect_docs, max_tokens=80, temperature=0.2)
        report['route'] = ans.get('route')
        report['answer_preview'] = ans.get('answer','')[:160]
        if ans.get('route') == 'failed':
            report['warning_generation'] = 'All SDK routes failed for sample query'
    except Exception as e:
        report['error_generation'] = str(e)
    return report

pprint.pprint(rag_self_test())

{'alt_model': 'deepseek-r1-distill-qwen-7b',
 'answer_preview': 'Okay, so I need to figure out why someone would use '
                   'Retrieval Augmented Generation (RAG) with local inference. '
                   'Let me start by understanding each part of the qu',
 'base': 'http://127.0.0.1:59778',
 'raw_model': 'deepseek-r1-distill-qwen-7b-cuda-gpu:0',
 'retrieved_indices': [0, 3, 1],
 'route': 'chat-first'}


### व्याख्या: ब्याच क्वेरी स्मोक टेस्ट
`answer()` मार्फत केही प्रतिनिधि प्रयोगकर्ता प्रश्नहरू चलाएर निम्न कुराहरू प्रमाणित गरिन्छ:
- पुनःप्राप्ति सूचकांकहरू सम्भावित समर्थन गर्ने कागजातहरूसँग मेल खान्छन्।
- फलब्याक राउटिङ काम गर्छ (राउट मान 'failed' छैन)।
- उत्तरहरूले आधारभूत निर्देशनको पालना गर्छन् (कुनै भ्रम सिर्जना गर्दैनन्)।
अन्तिम परिणाम वस्तुलाई अनियमित निरीक्षणको लागि कैद गरिन्छ।


In [16]:
# Quick test queries

queries = [

    "Why use RAG with local inference?",

    "What does vector similarity search do?",

    "Explain privacy benefits."

]



last_result = None

for q in queries:

    try:

        r = answer(q)

        last_result = r

        print(f"Q: {q}\nA: {r['answer']}\nDocs: {r['docs']}\n---")

    except Exception as e:

        print(f"Failed answering '{q}': {e}")



last_result

Q: Why use RAG with local inference?
A: Okay, so I need to figure out why someone would use Retrieval Augmented Generation (RAG) with local inference. Let me start by understanding each part of the question.

First, RAG. From the context given, Doc 1 says that RAG improves answer grounding by injecting relevant context. So RAG is a method that uses retrieval techniques to find the most relevant parts of a document or corpus to augment the generation process. This probably helps in making the generated answers more accurate because they're backed by real data.

Then, local inference. Doc 0 mentions that Foundry Local provides an OpenAI-compatible local inference endpoint. So local inference means running the model on the user's device rather than sending the request to a remote server. This is good for privacy and reducing latency, but it might have limitations in terms of model size or capabilities compared to cloud-based options.

Now, combining RAG with local inference. The context s

{'query': 'Explain privacy benefits.',
 'answer': 'Okay, so I need to explain the privacy benefits mentioned in the provided context. Let me look at the context again. The context includes three documents:\n\nDoc 2 says Edge AI reduces latency and preserves privacy via local execution.\nDoc 3 mentions Small Language Models can offer competitive quality with lower resource usage.\nDoc 1 states Retrieval Augmented Generation improves answer grounding by injecting relevant context.\n\nThe question is about explaining the privacy benefits. So, I should focus on the parts of the context that talk about privacy. \n\nLooking at Doc 2, it mentions Edge AI reduces latency and preserves privacy via local execution. That seems directly related to privacy. I think "local execution" means that the AI processes data on the device itself rather than sending it to a server. This could mean that data doesn\'t have to be transmitted, which might help protect user privacy because it avoids centralizing d

### व्याख्या: एकल उत्तर सुविधा कल
सजिलो प्रतिलिपि/पेस्ट प्रयोग वा पछि सन्दर्भको लागि अन्तिम छिटो एकल-प्रश्न कल। `answer()` को पूर्व-तयारी सोधपुछहरू पछि पुनः प्रयोगको लागि समान परिणाम सुनिश्चित गर्ने प्रदर्शन गर्दछ।


In [17]:
result = answer('Why use RAG with local inference?')
result

{'query': 'Why use RAG with local inference?',
 'answer': "Okay, so I need to figure out why someone would use Retrieval Augmented Generation (RAG) with local inference. Let me start by understanding each part of the question.\n\nFirst, RAG. From the context given, Doc 1 says that RAG improves answer grounding by injecting relevant context. So RAG is a method that uses retrieval techniques to find the most relevant parts of a document or corpus to augment the generation process. This probably helps in making the generated answers more accurate because they're backed by real data.\n\nThen, local inference. Doc 0 mentions that Foundry Local provides an OpenAI-compatible local inference endpoint. So local inference means running the model on the user's device rather than sending the request to a remote server. This is good for privacy and reducing latency, but it might have limitations in terms of model size or capabilities compared to cloud-based options.\n\nNow, combining RAG with local


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको हो। हामी यथासम्भव शुद्धता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। मूल दस्तावेज़ यसको मातृभाषामा आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
